# Table tennis simulation

This example is intended to show the usage of `Sim.jl`.

In [1]:
using Sim, Printf
import Sim.init!

We define some datastructures for states and events and players.

In [2]:
abstract type PState end
struct Idle <: PState end
struct Wait <: PState end
struct Unalert <: PState end

abstract type PEvent end
struct Start <: PEvent end
struct Serve <: PEvent end
struct Return <: PEvent end
struct Miss <: PEvent end

mutable struct Player
    name::AbstractString
    sim::Union{Number, Clock}
    opp::Union{Number,Player}
    state::PState
    accuracy::Float64
    attentiveness::Float64
    score::Int64

    Player(name, acc, att) = new(name, 0, 0, Idle(), acc, att, 0)
end

Then we have some physical facts to define and a function to randomize them:

In [3]:
const dist = 3 # distance for ball to fly [m]
const vs   = 10 # serve velocity [m/s]
const vr   = 20 # return velocity [m/s]

rd(s::Float64) = randn()*s + 1

rd (generic function with 1 method)

Next we must describe the behaviour of our players. Players are modeled as finite state machines, which have defined states and react to known events. This is done with the `step!` function.

In [4]:
"initialize a player"
function init!(p::Player, s::Clock, opp::Player)
    p.sim = s
    p.opp = opp
    if rand() ≤ p.attentiveness
        p.state = Wait()
    else
        p.state = Unalert()
    end
end

"a player serves a ball"
function serve(p::Player)
    ts = 3 + dist*rd(0.15)/(vs*rd(0.25))
    if rand() ≤ p.accuracy
        event!(p.sim, :(step!($(p.opp), Serve())), after, ts)
        @printf("%.2f: %s serves %s\n", p.sim.time+ts, p.name, p.opp.name)
    else
        event!(p.sim, :(step!($(p.opp), Miss())), after, ts)
        @printf("%.2f: %s serves and misses %s\n", p.sim.time+ts, p.name, p.opp.name)
    end
    if rand() ≥ p.attentiveness
        p.state = Unalert()
    end
end

"a player returns a ball"
function ret(p::Player)
    tr = dist*rd(0.15)/(vr*rd(0.25))
    if rand() ≤ p.accuracy
        event!(p.sim, :(step!($(p.opp), Return())), after, tr)
        @printf("%.2f: %s returns %s\n", p.sim.time+tr, p.name, p.opp.name)
    else
        event!(p.sim, :(step!($(p.opp), Miss())), after, tr)
        @printf("%.2f: %s returns and misses %s\n", p.sim.time+tr, p.name, p.opp.name)
    end
    if rand() ≥ p.attentiveness
        p.state = Unalert()
    end
end

"default transition for players"
step!(p::Player, q::PState, σ::PEvent) =
        println("undefined transition for $(p.name), $q, $σ")

"player p gets a start command"
step!(p::Player, ::Wait, ::Start) = serve(p)

"player p is waiting and gets served or returned"
step!(p::Player, ::Wait, ::Union{Serve, Return}) = ret(p)

"player p is unalert and gets served or returned"
function step!(p::Player, ::Unalert, ::Union{Serve, Return})
    @printf("%.2f: %s looses ball\n", p.sim.time, p.name)
    p.opp.score += 1
    p.state = Wait()
    serve(p)
end

"player p is waiting or unalert and gets missed"
function step!(p::Player, ::Union{Wait, Unalert}, ::Miss)
    p.score += 1
    p.state = Wait()
    serve(p)
end

"simplified `step!` call"
step!(p::Player, σ::PEvent) = step!(p, p.state, σ)

step!

In order to setup a simulation, we have to introduce a `Clock`, create and initialize the players, to start and to run the game:

In [5]:
sim = Clock()
ping = Player("Ping", 0.90, 0.90)
pong = Player("Pong", 0.90, 0.90)
init!(ping, sim, pong)
init!(pong, sim, ping)
step!(ping, Start())

run!(sim, 30)
println("Ping scored $(ping.score)")
println("Pong scored $(pong.score)")

3.30: Ping serves Pong
3.43: Pong returns Ping
3.59: Ping returns Pong
3.72: Pong returns Ping
3.86: Ping returns Pong
4.01: Pong returns Ping
4.18: Ping returns Pong
4.35: Pong returns Ping
4.45: Ping returns Pong
4.45: Pong looses ball
7.99: Pong serves Ping
7.99: Ping looses ball
11.15: Ping serves Pong
11.28: Pong returns Ping
11.38: Ping returns Pong
11.54: Pong returns Ping
11.71: Ping returns and misses Pong
14.97: Pong serves Ping
15.21: Ping returns Pong
15.42: Pong returns Ping
15.60: Ping returns Pong
15.77: Pong returns Ping
15.77: Ping looses ball
19.12: Ping serves Pong
19.23: Pong returns Ping
19.46: Ping returns and misses Pong
22.77: Pong serves Ping
22.92: Ping returns Pong
23.06: Pong returns Ping
23.19: Ping returns Pong
23.41: Pong returns and misses Ping
27.06: Ping serves Pong
27.18: Pong returns Ping
27.18: Ping looses ball
30.64: Ping serves Pong
Finished: 30 events, simulation time: 30.0
Ping scored 2
Pong scored 5
